# 68 - Magic 5-gon ring

## Problem Statement

<p>Consider the following "magic" 3-gon ring, filled with the numbers 1 to 6, and each line adding to nine.</p>
<div class="center">
<img src="0068_1.png" class="dark_img" alt=""><br></div>
<p>Working <b>clockwise</b>, and starting from the group of three with the numerically lowest external node (4,3,2 in this example), each solution can be described uniquely. For example, the above solution can be described by the set: 4,3,2; 6,2,1; 5,1,3.</p>
<p>It is possible to complete the ring with four different totals: 9, 10, 11, and 12. There are eight solutions in total.</p>
<div class="center">
<table width="400" cellspacing="0" cellpadding="0"><tr><td width="100"><b>Total</b></td><td width="300"><b>Solution Set</b></td>
</tr><tr><td>9</td><td>4,2,3; 5,3,1; 6,1,2</td>
</tr><tr><td>9</td><td>4,3,2; 6,2,1; 5,1,3</td>
</tr><tr><td>10</td><td>2,3,5; 4,5,1; 6,1,3</td>
</tr><tr><td>10</td><td>2,5,3; 6,3,1; 4,1,5</td>
</tr><tr><td>11</td><td>1,4,6; 3,6,2; 5,2,4</td>
</tr><tr><td>11</td><td>1,6,4; 5,4,2; 3,2,6</td>
</tr><tr><td>12</td><td>1,5,6; 2,6,4; 3,4,5</td>
</tr><tr><td>12</td><td>1,6,5; 3,5,4; 2,4,6</td>
</tr></table></div>
<p>By concatenating each group it is possible to form 9-digit strings; the maximum string for a 3-gon ring is 432621513.</p>
<p>Using the numbers 1 to 10, and depending on arrangements, it is possible to form 16- and 17-digit strings. What is the maximum <b>16-digit</b> string for a "magic" 5-gon ring?</p>
<div class="center">
<img src="0068_2.png" class="dark_img" alt=""><br></div>


## Solution

We can make a few observations:
- The last number of a row is the middle number of the next row
- The middle number of the first row is the last number of the last row
- Given the first and middle numbers, $x_1$ and $x_2$ in a row, and the sum of the previous row, $S_{i-1}$, the third number, $x_3$ is given by $x_3 = S_{i-1} - x_1 - x_2$. The solution only valid if $1 \leq x_3 \leq 10$.

Note that we can represent the $n$-gon as an $n \times 3$ matrix where each row represents 3 connected nodes starting from the external one. We initialize this matrix and then we simply fill the values using backtracking and taking into account the above observations. We keep a visited set with the digits already selected. Whenever we obtain a solution, we check that it has length 16 and check if it is greater than the previous maximum solution (if any).

The code below is not optimised but efficient enough given the small constraints.

In [6]:
import numpy as np

def reorder_matrix(matrix):
    min_index = np.argmin(matrix[:, 0])
    reordered_matrix = np.concatenate((matrix[min_index:], matrix[:min_index]))
    return reordered_matrix

n = 5
m = 3
max_digit = 10

matrix = np.zeros((n, m))
matrix = matrix.astype(int)

res = []
visited = set()

def solve(i, j):
    if i == n:
        mat = reorder_matrix(matrix)
        mat = "".join(mat.flatten().astype(str))
        if len(mat) == 16:
            if len(res) == 0:
                res.append(mat)
            else:
                res[0] = max(res[0], mat)
    if j == 0:
        if i == n - 1:
            target = np.sum(matrix[i-1, :]) - matrix[i, 2] - matrix[i, 1]
            if 0 < target <= max_digit and target not in visited:
                matrix[i][j] = target
                if i < n - 1:
                    matrix[i + 1][1] = target
                visited.add(target)
                solve(i+1, 0)
                visited.remove(target)

        else:
            for x in range(max_digit, 0, -1):
                if x in visited:
                    continue
                matrix[i, j] = x
                visited.add(x)
                if i == 0:
                    solve(i, 1)
                else:
                    solve(i, 2)
                visited.remove(x)
    elif j == 1:
        for x in range(max_digit, 0, -1):
            if x not in visited:
                matrix[i, j] = x
                matrix[-1, -1] = x
                visited.add(x)
                solve(i, 2)
                visited.remove(x)
    elif j == 2:
        if i == n - 1:
            solve(i + 1, 0)
        if i > 0:
            target = np.sum(matrix[i-1, :]) - matrix[i, 0] - matrix[i, 1]
            if 0 < target <= max_digit and target not in visited:
                matrix[i][j] = target
                if i < n - 1:
                    matrix[i + 1][1] = target
                visited.add(target)
                solve(i+1, 0)
                visited.remove(target)
        else:
            for x in range(max_digit, 0, -1):
                if x not in visited:
                    matrix[i, j] = x
                    matrix[i + 1, 1] = x
                    visited.add(x)
                    solve(i+1, 0)
                    visited.remove(x)

In [7]:
solve(0, 0)

res[0]

'6531031914842725'